In [21]:
# loading libraries 
import pandas as pd
from sklearn.preprocessing import StandardScaler


In [40]:
#read and display data
df1 =pd.read_csv("resources/NY-House-Dataset.csv")
df2 = pd.read_csv('./resources/housing_additional_data_file.csv')  # This file has lat and lon
print(display(df1.head(5)))
print(display(df2.head(5)))

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,LOCALITY,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.000000,1400.0,2 E 55th St Unit 803,"New York, NY 10022","2 E 55th St Unit 803New York, NY 10022",New York County,New York,Manhattan,East 55th Street,Regis Residence,"Regis Residence, 2 E 55th St #803, New York, N...",40.761255,-73.974483
1,Brokered by Serhant,Condo for sale,195000000,7,10.000000,17545.0,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",Central Park Tower Penthouse-217 W 57th New Yo...,United States,New York,New York County,New York,West 57th Street,"217 W 57th St, New York, NY 10019, USA",40.766393,-73.980991
2,Brokered by Sowae Corp,House for sale,260000,4,2.000000,2015.0,620 Sinclair Ave,"Staten Island, NY 10312","620 Sinclair AveStaten Island, NY 10312",United States,New York,Richmond County,Staten Island,Sinclair Avenue,"620 Sinclair Ave, Staten Island, NY 10312, USA",40.541805,-74.196109
3,Brokered by COMPASS,Condo for sale,69000,3,1.000000,445.0,2 E 55th St Unit 908W33,"Manhattan, NY 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,New York,New York County,New York,East 55th Street,"2 E 55th St, New York, NY 10022, USA",40.761398,-73.974613
4,Brokered by Sotheby's International Realty - E...,Townhouse for sale,55000000,7,2.373861,14175.0,5 E 64th St,"New York, NY 10065","5 E 64th StNew York, NY 10065",United States,New York,New York County,New York,East 64th Street,"5 E 64th St, New York, NY 10065, USA",40.767224,-73.969856


None


,lat,lon,suburb,postcode,category,result_type,state,city
0,40.761255,-73.974483,Manhattan,10022,catering.restaurant,amenity,New York,New York
1,40.766393,-73.980991,Manhattan,10019,building.residential,amenity,New York,New York
2,40.541805,-74.196109,Staten Island,10312,building.residential,building,New York,New York
3,40.761398,-73.974613,Manhattan,10022,catering.bar,amenity,New York,New York
4,40.767224,-73.969856,Manhattan,10065,building,building,New York,New York


None


In [38]:
df2

,lat,lon,suburb,postcode,category,result_type,state,city
0,40.761255,-73.974483,Manhattan,10022,catering.restaurant,amenity,New York,New York
1,40.766393,-73.980991,Manhattan,10019,building.residential,amenity,New York,New York
2,40.541805,-74.196109,Staten Island,10312,building.residential,building,New York,New York
3,40.761398,-73.974613,Manhattan,10022,catering.bar,amenity,New York,New York
4,40.767224,-73.969856,Manhattan,10065,building,building,New York,New York
...,...,...,...,...,...,...,...,...
4796,40.774350,-73.955879,Manhattan,10075,building,building,New York,New York
4797,40.732538,-73.860152,Queens,11374,building,building,New York,New York
4798,40.745882,-74.003398,Manhattan,10011,building,building,New York,New York
4799,40.742770,-73.872752,Elmhurst,11373,NaN,building,New York,New York


In [26]:
df1.columns

Index(['BROKERTITLE', 'TYPE', 'PRICE', 'BEDS', 'BATH', 'PROPERTYSQFT',
       'ADDRESS', 'STATE', 'MAIN_ADDRESS', 'ADMINISTRATIVE_AREA_LEVEL_2',
       'LOCALITY', 'SUBLOCALITY', 'STREET_NAME', 'LONG_NAME',
       'FORMATTED_ADDRESS', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

verifiying the api data

In [44]:
print(df2['suburb'].isnull().sum())  # Count missing sublocality values
print(df2['state'].value_counts())       # Check unique state values


0
state
New York    4801
Name: count, dtype: int64


In [45]:
# Clean sublocality and state columns in both datasets
df1['SUBLOCALITY'] = df1['SUBLOCALITY'].str.strip().str.lower()
df1['STATE'] = df1['STATE'].str.strip().str.lower()

df2['suburb'] = df2['suburb'].str.strip().str.lower()
df2['state'] = df2['state'].str.strip().str.lower()


In [46]:
# Drop duplicates in the key column
df1 = df1.drop_duplicates(subset=['LATITUDE', 'LONGITUDE'])
df2 = df2.drop_duplicates(subset=['lat', 'lon'])

In [47]:
# Merge the DataFrames using different column names
full_data_df = pd.merge(df1, df2, left_on=['LATITUDE', 'LONGITUDE'], right_on=['lat', 'lon'], how='inner')
# Display the merged DataFrame
full_data_df

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,...,LATITUDE,LONGITUDE,lat,lon,suburb,postcode,category,result_type,state,city
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.000000,1400.000000,2 E 55th St Unit 803,"new york, ny 10022","2 E 55th St Unit 803New York, NY 10022",New York County,...,40.761255,-73.974483,40.761255,-73.974483,manhattan,10022,catering.restaurant,amenity,new york,New York
1,Brokered by Serhant,Condo for sale,195000000,7,10.000000,17545.000000,Central Park Tower Penthouse-217 W 57th New Yo...,"new york, ny 10019",Central Park Tower Penthouse-217 W 57th New Yo...,United States,...,40.766393,-73.980991,40.766393,-73.980991,manhattan,10019,building.residential,amenity,new york,New York
2,Brokered by Sowae Corp,House for sale,260000,4,2.000000,2015.000000,620 Sinclair Ave,"staten island, ny 10312","620 Sinclair AveStaten Island, NY 10312",United States,...,40.541805,-74.196109,40.541805,-74.196109,staten island,10312,building.residential,building,new york,New York
3,Brokered by COMPASS,Condo for sale,69000,3,1.000000,445.000000,2 E 55th St Unit 908W33,"manhattan, ny 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,...,40.761398,-73.974613,40.761398,-73.974613,manhattan,10022,catering.bar,amenity,new york,New York
4,Brokered by Sotheby's International Realty - E...,Townhouse for sale,55000000,7,2.373861,14175.000000,5 E 64th St,"new york, ny 10065","5 E 64th StNew York, NY 10065",United States,...,40.767224,-73.969856,40.767224,-73.969856,manhattan,10065,building,building,new york,New York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,Brokered by Brown Harris Stevens,Condo for sale,945000,2,2.000000,903.000000,2351 Adam Clayton Powell Jr Blvd Apt 614,"manhattan, ny 10030",2351 Adam Clayton Powell Jr Blvd Apt 614Manhat...,New York,...,40.816711,-73.942050,40.816711,-73.942050,manhattan,10030,building,amenity,new york,New York
4196,Brokered by RE MAX Edge,Multi-family home for sale,2999999,15,9.000000,3600.000000,2825-2827 W 15th St,"brooklyn, ny 11224","2825-2827 W 15th StBrooklyn, NY 11224",United States,...,40.578450,-73.982739,40.578450,-73.982739,brooklyn,11224,building,building,new york,New York
4197,Brokered by COMPASS,Co-op for sale,599000,1,1.000000,2184.207862,222 E 80th St Apt 3A,"manhattan, ny 10075","222 E 80th St Apt 3AManhattan, NY 10075",New York,...,40.774350,-73.955879,40.774350,-73.955879,manhattan,10075,building,building,new york,New York
4198,Brokered by Douglas Elliman - 575 Madison Ave,Co-op for sale,1275000,1,1.000000,2184.207862,427 W 21st St Unit Garden,"new york, ny 10011","427 W 21st St Unit GardenNew York, NY 10011",United States,...,40.745882,-74.003398,40.745882,-74.003398,manhattan,10011,building,building,new york,New York


In [28]:
full_data_df.columns

Index(['BROKERTITLE', 'TYPE', 'PRICE', 'BEDS', 'BATH', 'PROPERTYSQFT',
       'ADDRESS', 'STATE', 'MAIN_ADDRESS', 'ADMINISTRATIVE_AREA_LEVEL_2',
       'LOCALITY', 'SUBLOCALITY', 'STREET_NAME', 'LONG_NAME',
       'FORMATTED_ADDRESS', 'LATITUDE', 'LONGITUDE', 'lat', 'lon', 'suburb',
       'postcode', 'category', 'result_type', 'state', 'city'],
      dtype='object')

In [72]:
# Selecting the Necessary Columns
selected_columns = [
    'BEDS', 'BATH', 'PROPERTYSQFT', 'TYPE',
    'state', 'city','suburb', 'LATITUDE', 'LONGITUDE','postcode'
]

# Subset the DataFrame
data = full_data_df[selected_columns]


In [73]:
data.head()

,BEDS,BATH,PROPERTYSQFT,TYPE,state,city,suburb,LATITUDE,LONGITUDE,postcode
0,2,2.000000,1400.0,Condo for sale,new york,New York,manhattan,40.761255,-73.974483,10022
1,7,10.000000,17545.0,Condo for sale,new york,New York,manhattan,40.766393,-73.980991,10019
2,4,2.000000,2015.0,House for sale,new york,New York,staten island,40.541805,-74.196109,10312
3,3,1.000000,445.0,Condo for sale,new york,New York,manhattan,40.761398,-73.974613,10022
4,7,2.373861,14175.0,Townhouse for sale,new york,New York,manhattan,40.767224,-73.969856,10065


In [66]:
# print(data['category'].unique())


In [67]:
# # Define residential categories
# housing_categories = [
#     'building.residential', 'accommodation.apartment', 'building.accommodation'
# ]

# # Filter the dataset for residential categories
# housing_df = data[data['category'].isin(housing_categories)]

# # Check the filtered data
# housing_df.shape
# housing_df.head()


In [ ]:
# housing_df.columns

Index(['PRICE', 'BEDS', 'BATH', 'PROPERTYSQFT', 'TYPE', 'state', 'city',
       'suburb', 'LATITUDE', 'LONGITUDE', 'postcode', 'category',
       'result_type'],
      dtype='object')

In [ ]:
# housing_df.shape

(826, 13)

In [59]:
# Check for Missing Values in the Selected Columns
housing_df.isnull().sum()



PRICE           0
BEDS            0
BATH            0
PROPERTYSQFT    0
TYPE            0
state           0
city            0
suburb          0
LATITUDE        0
LONGITUDE       0
postcode        0
category        0
result_type     0
dtype: int64

In [74]:
#Encode Categorical Variables
Encode = pd.get_dummies(data, columns=['TYPE', 'state', 'suburb'], drop_first=True)


In [75]:
Encode.head()

,BEDS,BATH,PROPERTYSQFT,city,LATITUDE,LONGITUDE,postcode,TYPE_Coming Soon,TYPE_Condo for sale,TYPE_Condop for sale,...,suburb_tribeca,suburb_two bridges,suburb_upper east side,suburb_upper west side,suburb_wakefield,suburb_washington heights,suburb_west village,suburb_whitestone,suburb_williamsburg,suburb_woodside
0,2,2.000000,1400.0,New York,40.761255,-73.974483,10022,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,7,10.000000,17545.0,New York,40.766393,-73.980991,10019,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,4,2.000000,2015.0,New York,40.541805,-74.196109,10312,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3,1.000000,445.0,New York,40.761398,-73.974613,10022,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,7,2.373861,14175.0,New York,40.767224,-73.969856,10065,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [76]:
# Encode Categorical Columns
scaler = StandardScaler()
numerical_features = ['BEDS', 'BATH', 'PROPERTYSQFT', 'LATITUDE', 'LONGITUDE']
Encode[numerical_features] = scaler.fit_transform(data[numerical_features])
